In [1]:
import tensorflow.contrib.slim as slim
import tensorflow as tf

import cifar10_input

batch_size = 64
images_trains, labels_trains = cifar10_input.distorted_inputs(data_dir="../cifar_data/cifar-10-batches-bin/",
                                                              batch_size=batch_size)

images_test, labels_test = cifar10_input.inputs(eval_data=True,data_dir="../cifar_data/cifar-10-batches-bin/",
                                               batch_size=batch_size)
def alexnet_arg_scope():
    with slim.arg_scope([slim.conv2d, slim.fully_connected], activation_fn = tf.nn.relu, 
                        weights_initializer = tf.truncated_normal_initializer(stddev=1e-2)):
        with slim.arg_scope([slim.conv2d, slim.max_pool2d], padding='VALID') as sc:
            return sc

def alexnet(inputs, reuse=None):
    with tf.variable_scope('AlexNet', reuse=reuse):
        net = slim.conv2d(inputs, 64, [5,5], scope='conv1')
        net = slim.max_pool2d(net, [3,3], scope='pool1')
        net = slim.conv2d(net, 64, [5,5], scope='conv2')
        net = slim.max_pool2d(net, [3,3], scope='pool2')
        net = slim.flatten(net)
        net = slim.fully_connected(net, 384,scope='fc3')
        net = slim.fully_connected(net, 192, scope='fc4')
        net = slim.fully_connected(net, 10, activation_fn=None, scope='classification')
        
        return net

image_holder = tf.placeholder(shape=[batch_size,24,24,3], dtype=tf.float32)
label_holder = tf.placeholder(shape=[batch_size],dtype=tf.int32)

with slim.arg_scope(alexnet_arg_scope()):
    train_out = alexnet(image_holder)

var_size, var_bytes = slim.model_analyzer.analyze_vars(variables=tf.model_variables(),print_info=True)

with tf.variable_scope('loss'):
    train_loss = tf.losses.sparse_softmax_cross_entropy(logits=train_out, labels=label_holder, scope='train')

with tf.variable_scope('accuracy'):
    train_acc = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(train_out, axis=-1, output_type=tf.int32), 
                                                label_holder), tf.float32))

lr = 1e-3
train_op = tf.train.MomentumOptimizer(learning_rate=lr, momentum=0.9).minimize(train_loss)



Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(tensor).repeat(num_epochs)`.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.FixedLengthRecordDataset`.
Filling queue with 20000 CIF

In [3]:
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()
tf.train.start_queue_runners()
for i in range(1000):
    image_bath, label_batch = sess.run([images_trains, labels_trains])
    _, loss1, acc1 = sess.run([train_op,train_loss,train_acc], feed_dict={image_holder: image_bath, label_holder: label_batch})
    if i % 10 ==0:
        print("Step%d  train loss %.6f train acc: %.6f" % (i+1, loss1, acc1))

/usr/local/lib/python3.5/dist-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


Step1  train loss 2.302600 train acc: 0.109375
Step11  train loss 2.302638 train acc: 0.093750
Step21  train loss 2.302575 train acc: 0.093750
Step31  train loss 2.302807 train acc: 0.031250
Step41  train loss 2.302336 train acc: 0.109375
Step51  train loss 2.302704 train acc: 0.125000
Step61  train loss 2.302702 train acc: 0.078125
Step71  train loss 2.302559 train acc: 0.093750
Step81  train loss 2.302287 train acc: 0.171875
Step91  train loss 2.302702 train acc: 0.109375
Step101  train loss 2.302760 train acc: 0.093750
Step111  train loss 2.302745 train acc: 0.062500
Step121  train loss 2.302089 train acc: 0.109375
Step131  train loss 2.302537 train acc: 0.093750


KeyboardInterrupt: 

In [8]:
import time
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()
tf.train.start_queue_runners()

for step in range(1000):
    start_time = time.time()
    image_batch, label_batch = sess.run([images_trains, labels_trains])
    _, loss_value = sess.run([train_op, train_loss],feed_dict={image_holder: image_batch, label_holder: label_batch})
    duration = time.time() - start_time
    if step % 10 ==0:
        examples_per_sec = batch_size/duration
        sec_per_batch = float(duration)
        
        format_str = ('step %d, loss=%.2f (%.1f examples/sec; %.3f sec/batch)')
        print(format_str % (step, loss_value, examples_per_sec, sec_per_batch))

/usr/local/lib/python3.5/dist-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


step 0, loss=2.30 (24.2 examples/sec; 2.648 sec/batch)
step 10, loss=2.30 (1290.2 examples/sec; 0.050 sec/batch)
step 20, loss=2.30 (1259.3 examples/sec; 0.051 sec/batch)
step 30, loss=2.30 (1025.0 examples/sec; 0.062 sec/batch)
step 40, loss=2.30 (1038.4 examples/sec; 0.062 sec/batch)
step 50, loss=2.30 (1303.6 examples/sec; 0.049 sec/batch)
step 60, loss=2.30 (1297.3 examples/sec; 0.049 sec/batch)
step 70, loss=2.30 (1019.7 examples/sec; 0.063 sec/batch)
step 80, loss=2.30 (1257.4 examples/sec; 0.051 sec/batch)
step 90, loss=2.30 (1305.8 examples/sec; 0.049 sec/batch)
step 100, loss=2.30 (816.7 examples/sec; 0.078 sec/batch)
step 110, loss=2.30 (935.8 examples/sec; 0.068 sec/batch)
step 120, loss=2.30 (1273.4 examples/sec; 0.050 sec/batch)


KeyboardInterrupt: 